### Automated subtraction of water spectra from infrared spectra of blood plasma

- two options are available for fitting in the specified region of interest:
    - linear regression
    - least squares

- requirements/assumptions:
    - every blood plasma sample spectrum has a suffix "_sample" (e.g., H002_sample) and has its own water spectrum named with "_water" suffix (e.g., H002_water)
    - currently works only with .csv files

In [ ]:
import os
import csv
import logging
from typing import Tuple, List, Literal, Union
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression
from pathlib import Path
import time
import plotly.graph_objects as go
import re
import glob

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

In [ ]:
# LINEAR REGRESSION FUNCTION
def linear_regression(wavenumber: pd.Series, 
                      y_sample: pd.Series,
                      y_reference: pd.Series,
                      region: Tuple[int, int],
                      k_initial_vals: Tuple[float, float] = (0.5, 1.5),
                      refinement_steps: List[float] = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
                      ) -> Tuple[pd.Series, pd.Series, float]:
                          
    """
    Performs linear regression to optimize the subtraction factor of reference/solvent from sample in a specified 
    region of interest by iteratively selecting the best 'k' to minimize slope.
    
    Args:
        wavenumber (pd.Series): Wavenumber values.
        y_sample (pd.Series): Sample spectrum.
        y_reference (pd.Series): Reference/solvent spectrum to subtract.
        region (Tuple[int, int]): Tuple of (start_index, end_index) in the wavenumber range for optimization.
        k_initial_vals (Tuple[float, float]): Initial min/max bounds for k coefficient.
        refinement_steps (List[float]): Step sizes for iterative refinement of k.

    Returns:
        Tuple[pd.Series, pd.Series, float]: wavenumber, corrected_spectrum, optimal coefficient k
    """
    start_index, end_index = region
    x_region = wavenumber.iloc[start_index:end_index].values.reshape(-1, 1)

    if x_region.size == 0:
        raise ValueError("Region of interest is empty or out of bounds.")

    def regression_slope(k: float) -> float:
        """Calculates slope from linear regression after solvent subtraction."""
        y_corrected = y_sample - k * y_reference
        y_region = y_corrected.iloc[start_index:end_index].values
        if y_region.size == 0:
            return np.inf
        model = LinearRegression().fit(x_region, y_region)
        return abs(model.coef_[0])

    k_min, k_max = k_initial_vals

    for step in refinement_steps:
        k_values = np.arange(k_min, k_max, step)
        coefs = [(k, regression_slope(k)) for k in k_values]

        if len(coefs) < 3:
            continue

        sorted_coefs = sorted(coefs, key=lambda x: x[1])
        k_min, k_max = sorted([sorted_coefs[1][0], sorted_coefs[2][0]])

    optimal_k = k_min
    corrected_spectrum = y_sample - optimal_k * y_reference

    return wavenumber, corrected_spectrum, optimal_k

In [ ]:
# LEAST SQUARES FUNCTION
def least_squares_subtraction(
    wavenumber: pd.Series,
    y_sample: pd.Series,
    y_reference: pd.Series,
    region: Tuple[int, int],
    initial_val: float = 1.0,
    ) -> Tuple[pd.Series, pd.Series, float]:
    """
    Minimizes the squared residuals between sample and scaled reference spectrum over a specific region.

    Args:
        wavenumber (pd.Series): Wavenumber values.
        y_sample (pd.Series): Sample spectrum.
        y_reference (pd.Series): Reference/solvent spectrum to subtract.
        region (Tuple[int, int]): Tuple of (start_index, end_index) in the wavenumber range for optimization.
        initial_val (float): Initial guess for the scaling factor (coefficient k).

    Returns:
        Tuple[pd.Series, pd.Series, float]: wavenumber, corrected spectrum, optimal coefficient k.
    """
    def least_squares(k: float) -> float:
        residual = y_sample - k * y_reference
        return np.sum(residual[region[0]:region[1]] ** 2)

    result = minimize(
        fun=least_squares,
        x0=initial_val,
        method='Nelder-Mead',
        options={'xatol': 1e-10, 'disp': False}
    )

    optimal_k = result.x[0]
    corrected = y_sample - optimal_k * y_reference
    return wavenumber, corrected, optimal_k

In [ ]:
def process_spectrum(
    sample_file: str,
    water_file: str,
    region: Tuple[int, int] = (52, 131),
    method: Literal['least_squares', 'linear_regression'] = 'least_squares'
    ) -> Tuple[pd.Series, pd.Series, float]:
    """
    Processes a sample and reference spectrum file and applies water subtraction.

    Args:
        sample_file (str): Path to sample spectrum CSV.
        water_file (str): Path to reference (water) spectrum CSV.
        region (Tuple[int, int]): Tuple of (start_index, end_index) in the wavenumber range for optimization.
        method (str): Subtraction method: 'least_squares' or 'linear_regression'.

    Returns:
        Tuple[pd.Series, pd.Series, float]: wavenumbers, corrected sample, and subtraction coefficient (k).
    """
    try:
        sample_df = pd.read_csv(sample_file, header=None, delimiter=";", decimal=",", skiprows=130)
        reference_df = pd.read_csv(water_file, header=None, delimiter=";", decimal=",", skiprows=130)
        wavenumber = sample_df.iloc[:, 0]
        sample = sample_df.iloc[:, 1]
        reference = reference_df.iloc[:, 1]

        if method == 'least_squares':
            return least_squares_subtraction(wavenumber, sample, reference, region)
        elif method == 'linear_regression':
            return linear_regression(wavenumber, sample, reference, region)
        else:
            raise ValueError("Unsupported method. Use 'least_squares' or 'linear_regression'.")
    except Exception as e:
        raise RuntimeError(f"Error processing files '{sample_file}' and '{water_file}': {e}")


In [ ]:
def folder_reference_subtraction(
    folder_path: str,
    output_folder: str,
    region: Tuple[int, int] = (52, 131),
    method: Literal['least_squares', 'linear_regression'] = 'least_squares',
    file_suffixes: Tuple[str, str] = ('sample', 'water'),
    verbose: bool = True
    ) -> None:
    """
    Performs reference/solvent subtraction for all pairs of sample-reference spectra files.

    Args:
        folder_path (str): Input folder with spectra files.
        output_folder (str): Output folder for processed files.
        region (Tuple[int, int]): Tuple of (start_index, end_index) in the wavenumber range for optimization.
        method (str): Subtraction method. 'least_squares' or 'linear_regression'.
        file_suffixes (Tuple[str, str]): Tuple defining sample and reference file suffixes.
        verbose (bool): If True, prints progress and errors.
    """
    os.makedirs(output_folder, exist_ok=True)
    files = sorted(os.listdir(folder_path))
    sample_files = [f for f in files if file_suffixes[0] in f]
    reference_files = [f for f in files if file_suffixes[1] in f]
 
    # removal of file suffixes to match sample and reference files 
    def strip_file_suffix(filename: str, suffixes: Tuple[str, str]) -> str:
        stem = Path(filename).stem
        for suffix in suffixes:
            if stem.endswith(suffix):
                return stem[: -len(suffix)]
        return stem

    sample_dict = {strip_file_suffix(f, file_suffixes): f for f in sample_files}
    reference_dict = {strip_file_suffix(f, file_suffixes): f for f in reference_files}

    common_keys = sorted(set(sample_dict.keys()) & set(reference_dict.keys()))
    if not common_keys:
        raise ValueError("No matching sample-reference file pairs found based on name suffixes.")

    paired_files = [(sample_dict[key], reference_dict[key]) for key in common_keys]

    if verbose:
        unmatched_samples = set(sample_dict.keys()) - set(reference_dict.keys())
        unmatched_references = set(reference_dict.keys()) - set(sample_dict.keys())

        if unmatched_samples:
            print("Unmatched sample files:", unmatched_samples)
        if unmatched_references:
            print("Unmatched reference files:", unmatched_references)
       
    for sample_name, reference_name in paired_files:
        sample_path = os.path.join(folder_path, sample_name)
        reference_path = os.path.join(folder_path, reference_name)

        try:
            x_sample, corrected_spectrum, best_k = process_spectrum(
                sample_path, reference_path,
                region=region,
                method=method
            )

            output_filename = f"{Path(sample_name).stem}_water_subtract.txt"
            output_path = os.path.join(output_folder, output_filename)

            pd.concat([x_sample, corrected_spectrum], axis=1).to_csv(
                output_path, index=False, header=False, sep=";"
            )

            if verbose:
                print(f"Processed: {sample_name} | k = {best_k:.5f}")

        except Exception as e:
            if verbose:
                print(f"Error processing {sample_name} & {reference_name}: {e}")

In [ ]:
def read_and_combine_txt_files(
    output_folder: str,
    file_suffix: str = "_water_subtract.txt",
    output_filename: str = "0_water_subtraction_output.csv",
    delimiter: str = ";",
    include_first_column: bool = True,
    sort_files: bool = True
    ) -> str:
    """
    Combines .txt files after water subtraction into a single CSV.

    Args:
        output_folder (str): Folder containing processed .txt files.
        file_suffix (str): Suffix to identify relevant files.
        output_filename (str): Filename for the combined CSV output.
        delimiter (str): Delimiter used in the .txt files.
        include_first_column (bool): Whether to include the x-axis (wavenumber) 
                                    from the first file (assuming it is the first column).
        sort_files (bool): Whether to alphabetically sort the input files.

    Returns:
        str: Path to the combined CSV file.
    """
    txt_files = [f for f in os.listdir(output_folder) if f.endswith(file_suffix)]
    if sort_files:
        txt_files.sort()

    data = {}
    first_column = []

    for i, filename in enumerate(txt_files):
        file_path = os.path.join(output_folder, filename)
        y_values = []

        with open(file_path, 'r') as file:
            for line in file:
                columns = line.strip().split(delimiter)
                if len(columns) >= 2:
                    y_values.append(columns[1])
                    if i == 0 and include_first_column:
                        first_column.append(columns[0])

        data[Path(filename).stem] = y_values

    if include_first_column:
        data = {"Wavenumber": first_column, **data}

    output_path = os.path.join(output_folder, output_filename)

    names = list(data.keys())
    num_rows = len(next(iter(data.values())))
    
    with open(output_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(names)

        for row_index in range(num_rows):
            writer.writerow([data[col][row_index] for col in names])

    return output_path

In [ ]:
# WATER VAPOR SPECTRUM SUBTRACTION
"""
@author: Piotr Bruzdziak
Department of Physical Chemistry
Gdansk University of Technology
e-mail: piotr.bruzdziak@pg.edu.pl

This script allows to fit and subtract vapor spectra from a series of raw 
measured spectra. Feel free to use or modify the script, but please cite the 
paper where the method has been described:
    
How to use:

obj_spec,corr_spectra = atm_subtraction(wavenb,spectra_to_subtract,atm_spectra)

The main fauntion to use is atm_subtraction. It employs AtmFitParams class 
to perform the fitting of a spectrum with vapor spectra. 

atm_subtraction arguments:
    wavenb          - wavenumbers corresponding to raw spectra
    spectra_to_subtract - numpy array of raw spectra (columnwise)
    atm_spectra     - numpy array of vapor spectra (columnwise)
    additional keywords:
      SG_poly=3,SG_points=21 - parameters of Savitzky-Golay smoothing algorithm
    
    wavenumbers, raw and vapor spectra must have the same number of rows!
    
This function returns:

corr_spectra        - numpy array of all vapor corrected spectra (columnwise)    
obj_spec            - a list of spectra objects with the following useful 
                      properties and methods:
    .fit_atm_params - subtraction coefficients for consecutive vapor spectra
                      (last three coefficients correspond to the baseline!)
    .sub_spectrum   - vapor-corrected spectrum
    .plot()         - plots raw and corrected spectra 
					 (saving plot to file keywords: save=True, 
													filename='figure_name.png')
        
    example:
        obj_spectra[0].fit_atm_params - will produce subtraction (and baseline)
                                        coeffcients of the first (0) spectrum

"""
import numpy as np
from scipy.signal import savgol_filter as savitzky_golay
from scipy.optimize import least_squares as leastsq


def atm_subtraction(wavenb,spectra_to_subtract,atm_spectra,SG_poly=3,\
                    SG_points=21):
    spectra_to_subtract = np.asarray(spectra_to_subtract)
    list_of_spectra = []
    spectra_corrected = []
    if len(spectra_to_subtract.shape) == 1:
        no_of_spectra = 1
        list_of_spectra.append(AtmFitParams(wavenb,spectra_to_subtract,\
                                                atm_spectra,SG_poly=SG_poly,\
                                                SG_points=SG_points))
        spectra_corrected.append(list_of_spectra[0].sub_spectrum)
    else:
        no_of_spectra = spectra_to_subtract.shape[1]
        for i in range(no_of_spectra):
            list_of_spectra.append(AtmFitParams(wavenb,spectra_to_subtract[:,i],\
                                                atm_spectra,SG_poly=SG_poly,\
                                                SG_points=SG_points))
            spectra_corrected.append(list_of_spectra[i].sub_spectrum)
    return list_of_spectra,np.asarray(spectra_corrected).T
    

class AtmFitParams:    
    def __init__(self,wavenb,spectrum,atm_spectra,init_factor=10.0,\
                 SG_poly=5,SG_points=3):
        # list of wavenumbers
        self.wavenb = np.asarray(wavenb,dtype=float)
        # spectrum from which atmosphere (atm) spectra will be subtracted
        self.spectrum = np.asarray(spectrum,dtype=float)
        # array of atmosphere spectra
        self.atm_spectra = np.asarray(atm_spectra,dtype=float)
        if len(self.atm_spectra.shape) == 1:
            self.no_of_atm_spctr = 1
        else:
            self.no_of_atm_spctr = self.atm_spectra.shape[1]
        # initial multiplying factor for atm spectra
        self.init_factor = init_factor
        # initial quadratic baseline parameters
        bln_params = [0.0, 0.0, 0.0]
        # initial parameters of atm fitting
        self.init_fit_params = np.asarray((self.no_of_atm_spctr*\
                                           [self.init_factor] + bln_params),\
                                            dtype=float)
        no_of_parameters = np.size(self.init_fit_params)
        # boundaries for all parameters (can be changed)
        # set to +/- inf for atm intensities and +/-0.001 - 0.1 for baseline
        bln_bounds = [[-0.001,-0.01,-0.1],[0.001,0.01,0.1]]
        self.bounds = (no_of_parameters*[-np.inf],no_of_parameters*[np.inf])
        self.bounds[0][-3:] = bln_bounds[0]
        self.bounds[1][-3:] = bln_bounds[1]
        # Savitzky-Golay smoothing factors
        self.SG_poly = SG_poly
        self.SG_points = SG_points
        # Fitted parameters of water vapor subtraction (baseline - last 3)
        self.fit_atm_params = self.fit()
        # Vapor-corrected spectrum
        self.sub_spectrum = self.atm_subtract()   
    def baseline(self,bln_params):
        a = bln_params[0] #ax^2
        b = bln_params[1] #bx
        c = bln_params[2] #c
        return a*((self.wavenb)**2) + b*(self.wavenb) + c    
    def residuals(self,params,spectrum,wavenb,atm_spectra):
        baseln = self.baseline(params[-3:])
        if len(self.atm_spectra.shape) == 1:
            atm_sum = params[:-3]*atm_spectra
        else:
            atm_sum = np.sum((params[:-3]*atm_spectra),axis=1)
        total_sum = spectrum - atm_sum
        smoothed_total_sum = savitzky_golay(total_sum,self.SG_points,\
                                            self.SG_poly)
        return smoothed_total_sum - total_sum + baseln
    def fit(self):
        fit_result = leastsq(self.residuals,self.init_fit_params,args=\
                             (self.spectrum,self.wavenb,self.atm_spectra),\
                             ftol=1.49012e-10, xtol=1.49012e-10,\
                             bounds=self.bounds)
        fit_params = fit_result.x
        return fit_params
    def atm_subtract(self):
        baseln = self.baseline(self.fit_atm_params[-3:])
        if len(self.atm_spectra.shape) == 1:
            atm_sum = self.fit_atm_params[:-3]*self.atm_spectra
        else:
            atm_sum = np.sum((self.fit_atm_params[:-3]*self.atm_spectra),axis=1)
        sub_spectrum = self.spectrum - atm_sum - baseln
        return sub_spectrum

In [ ]:
def folder_water_vapor_subtraction(
    folder_path: str,
    output_folder: str,
    atm_spectra_file: Union[str, os.PathLike]
    ) -> None:
    """
    Performs water vapor spectra subtraction from all .txt files in a folder.

    Args:
        folder_path (str): Folder with spectra files.
        output_folder (str): Output folder for processed files.
        atm_spectra_file (str or PathLike): File path to the reference file containing water vapor spectra.
    """
    os.makedirs(output_folder, exist_ok=True)

    try:
        atm_spectra = pd.read_csv(atm_spectra_file, delimiter=";", decimal=".", skiprows=1, header=None)
        atm_spectra_values = atm_spectra.iloc[:, 1:].values
    except Exception as e:
        raise RuntimeError(f"Error reading atmospheric spectra file: {atm_spectra_file}\n{e}")

    input_files = sorted(glob.glob(os.path.join(folder_path, '*.txt')))

    for file_path in input_files:
        try:
            data = pd.read_csv(file_path, delimiter=";", decimal=".", header=None)
            wavenumbers = data.iloc[:, 0].values
            spectrum = data.iloc[:, 1].values

            _, corrected_spectrum = atm_subtraction(wavenumbers, spectrum, atm_spectra_values)

            output_filename = f"{os.path.splitext(os.path.basename(file_path))[0]}_vapor_subtract.txt"
            output_path = os.path.join(output_folder, output_filename)

            corrected_series = pd.Series(corrected_spectrum.ravel())
            pd.concat([pd.Series(wavenumbers), corrected_series], axis=1).to_csv(output_path, sep=';', index=False, header=False)

        except Exception as e:
            print(f"[ERROR] Failed to process file '{file_path}': {e}")


In [ ]:
def clean_spectrum_name(name: str) -> str:
    """
    Removes suffixes from spectra for plotting purposes
        
    """
    cleaned_name = re.sub(r'_?sample_water_subtract(_vapor_subtract)?', '', name, flags=re.IGNORECASE)
    return cleaned_name.strip("_")

def plot_spectra_from_csv(csv_file: str, 
                          wavenumber_range: tuple = None, 
                          title: str = None,
                          skiprows: int = 0,
                          ) -> None:
    """
    Plots spectra from a CSV file using Plotly.
    
    Handles both combined multi-spectra CSVs and raw two-column spectra files.
    
    Args:
        csv_file (str): Path to the CSV file.
        wavenumber_range (tuple, optional): Tuple specifying the (min, max) x-axis limits.
        skiprows (int, optional): Number of rows to skip at the start (e.g., metadata).
    """
    try:
        df = pd.read_csv(csv_file, skiprows=skiprows, header=None if skiprows else 'infer')
    except Exception as e:
        print(f"[ERROR] Could not read file: {e}")
        return   
    
    if df.empty or df.shape[1] < 2:
        print("[WARNING] CSV file is empty or has insufficient columns.")
        return

    fig = go.Figure()

    if df.shape[1] == 2:
        x = pd.to_numeric(df.iloc[:, 0], errors='coerce')
        y = pd.to_numeric(df.iloc[:, 1], errors='coerce')
        spectrum_name = os.path.splitext(os.path.basename(csv_file))[0]
        fig.add_trace(go.Scatter(
            x=x,
            y=y,
            mode='lines',
            name=spectrum_name,
            hovertemplate=(
                "<b>%{[spectrum_name]}</b><br>" +
                "wavenumber: %{x:.2f} cm⁻¹<br>" +
                "Absorbance: %{y:.5f}<extra></extra>"
            )
        ))
    else:
        x = pd.to_numeric(df.iloc[:, 0], errors='coerce')

        for i, spectrum in enumerate(df.columns[1:]):
            spectrum_name = df.columns[i+1] if isinstance(df.columns[i+1], str) else f"Spectrum {i+1}"
            cleaned_name = clean_spectrum_name(spectrum_name)
            y = pd.to_numeric(df.iloc[:, i+1], errors='coerce')

            fig.add_trace(go.Scatter(
                x=x,
                y=y,
                mode='lines',
                name=cleaned_name,
                hovertemplate=(
                    "<b>%{text}</b><br>" +
                    "Wavenumber: %{x:.2f} cm⁻¹<br>" +
                    "Absorbance: %{y:.5f}<extra></extra>"
                ),
                text=[cleaned_name]*len(x)
            ))

    fig.update_layout(
        title=title or "Spectrum",
        xaxis_title="Wavenumber (cm⁻¹)",
        yaxis_title="Absorbance",
        hovermode="closest",
        template="plotly_dark",
        height=600
    )

    if wavenumber_range:
        fig.update_xaxes(range=wavenumber_range)

    fig.show()
    
    
    
def plot_multiple_txt_spectra(folder: str,
                              skiprows: int = 0,
                              delimiter: str = ";",
                              decimal: str = ",",
                              file_extension: str = ".csv",
                              wavenumber_range: tuple = None,
                              title: str = None,
                              ) -> None:
    """
    Plots all files from a folder in a single plot. Assumes each file has two columns, one with wavenumbers
    and the other with absorbance values.

    Args:
        folder (str): Path to the folder containing spectra files.
        skiprows (int): Number of initial rows to skip in each file.
        delimiter (str): Column delimiter in the files.
        file_extension (str): File extension to include (e.g., '.csv' or '.txt').
        wavenumber_range (tuple, optional): Tuple specifying the (min, max) x-axis limits.
    """
    fig = go.Figure()

    files = [f for f in os.listdir(folder) if f.endswith(file_extension)]
    if not files:
        print("[WARNING] No matching files found.")
        return

    for filename in files:
        path = os.path.join(folder, filename)
        try:
            df = pd.read_csv(path, skiprows=skiprows, delimiter=delimiter, decimal=decimal)
            if df.shape[1] < 2:
                print(f"[WARNING] File {filename} has fewer than 2 columns. Skipping.")
                continue

            x = df.iloc[:, 0]
            y = df.iloc[:, 1]
            spectrum_name = Path(filename).stem

            fig.add_trace(go.Scatter(
                x=x,
                y=y,
                mode='lines',
                name=spectrum_name,
                hovertemplate=(
                    "<b>%{text}</b><br>" +
                    "wavenumber: %{x:.2f} cm⁻¹<br>" +
                    "Absorbance: %{y:.5f}<extra></extra>"
                ),
                text=[spectrum_name]*len(x)
            ))
        except Exception as e:
            print(f"[ERROR] Could not read {filename}: {e}")

    fig.update_layout(
        title=title,
        xaxis_title="wavenumber (cm⁻¹)",
        yaxis_title="Absorbance",
        hovermode="closest",
        template="plotly_dark",
        height=600
    )

    if wavenumber_range:
        fig.update_xaxes(range=wavenumber_range)

    fig.show()


In [ ]:
def main(
    INPUT_PATH: str,
    OUTPUT_PATH: str,
    WATER_VAPOR: str,
    REGION: Tuple[int, int] = (52, 131),
    METHOD: str = "least_squares",
    plot_raw_sample: bool = False,
    plot_reference: bool = False,
    plot_water_subtracted: bool = False,
    plot_vapor_subtracted: bool = False
    ):
    
    """
    Main pipeline to perform water and water vapor subtraction on spectral data.
    
    Args:
        INPUT_PATH (str): Input folder with spectra.
        OUTPUT_PATH (str): Output folder for processed spectra.
        WATER_VAPOR (str): File path to water vapor spectra.
        REGION (tuple): Tuple of (start_index, end_index) for optimization.
        METHOD (str): Subtraction method: 'least_squares' or 'linear_regression'.
    """
    
    start_time = time.time()

    if not os.path.exists(INPUT_PATH):
        print("[ERROR] Invalid input folder path.")
        return
    if not os.path.exists(WATER_VAPOR):
        print("[ERROR] Atmospheric spectra file not found.")
        return


    water_subtraction_folder = os.path.join(OUTPUT_PATH, "water_subtraction")
    vapor_subtraction_folder = os.path.join(OUTPUT_PATH, "vapor_subtraction")
    os.makedirs(OUTPUT_PATH, exist_ok=True)

    try:
        # subtracting reference/solvent spectra
        folder_reference_subtraction(
            INPUT_PATH,
            water_subtraction_folder,
            region=REGION,
            method=METHOD
        )
        
        # combining individual spectra into one .csv file
        water_combined_csv = read_and_combine_txt_files(
            water_subtraction_folder,
            file_suffix="_water_subtract.txt",
            output_filename=f"0_water_subtraction_output_{METHOD}.csv"
        )
        
        # subtracting water vapor spectra
        folder_water_vapor_subtraction(
            water_subtraction_folder,
            vapor_subtraction_folder,
            WATER_VAPOR
        )

        # combining individual spectra into one .csv file
        vapor_combined_csv = read_and_combine_txt_files(
            vapor_subtraction_folder,
            file_suffix="_vapor_subtract.txt",
            output_filename=f"0_vapor_subtraction_output_{METHOD}.csv"
        )

        if plot_raw_sample:
            plot_multiple_txt_spectra(
                folder=INPUT_PATH,
                skiprows=130,
                file_extension="_sample.CSV",
                delimiter=";",
                wavenumber_range=(4000, 650),
                title = "raw sample spectra"
            )
                    
        if plot_reference:
            plot_multiple_txt_spectra(
                folder=INPUT_PATH,
                skiprows=130,
                file_extension="_water.CSV",
                delimiter=";",
                wavenumber_range=(4000, 650),
                title = "raw reference spectra"
            )
                    
        if plot_water_subtracted:
            print("[INFO] Plotting water-subtracted spectra")
            plot_spectra_from_csv(water_combined_csv, 
                                  wavenumber_range=(1800, 1000), 
                                  title="spectra after water subtraction")

        if plot_vapor_subtracted:
            print("[INFO] Plotting vapor-subtracted spectra")
            plot_spectra_from_csv(vapor_combined_csv, 
                                  wavenumber_range=(1800, 1000), 
                                  title="spectra after water vapor subtraction")

        else:
            print("[INFO] No plots requested. Skipping plotting step.")
        end_time = time.time()
        print(f"[INFO] Finished in {end_time - start_time:.2f} seconds.")

    except Exception as e:
        print(f"[ERROR] An unexpected error occurred: {e}")




In [ ]:
# ======================
INPUT_FOLDER_PATH = "path_to_files"                 # Folder containing raw spectra files (.csv)
OUTPUT_FOLDER_PATH = "output_folder_path"           # Folder to save processed outputs
ATM_SPECTRA_FILE = "path_to_water_vapor_spectra"    # Path to water vapor spectra file (.csv)

REGION = (  ,  )                                    # Index range for subtraction optimization
METHOD = "least_squares"                            # "least_squares" or "linear_regression"
# ======================

main(
    INPUT_PATH=INPUT_FOLDER_PATH,
    OUTPUT_PATH=OUTPUT_FOLDER_PATH,
    WATER_VAPOR=ATM_SPECTRA_FILE,
    plot_raw_sample=True,
    plot_reference=True,
    plot_water_subtracted=True,
    plot_vapor_subtracted=True
    )